In [14]:
import pandas as pd
import numpy as np

In [15]:
bakery_sales = pd.read_csv('Bakery_sales.csv')
Coffee_sales = pd.read_csv('Coffee_Shop_sales.csv')

In [16]:
#clean the unit_price column by removing the € sign and converting the column to float
bakery_sales["unit_price"] = (
    bakery_sales["unit_price"]
    .str.replace("€", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.replace(" ", "", regex=False)
    .str.replace('"', "", regex=False)
    .str.strip()
    .astype(float)
)
#rename the columns to match the target schema
bakery_sales = bakery_sales.rename(
    columns={
        "ticket_number": "transaction_id",
        "date": "transaction_date",
        "time": "transaction_time",
        "Quantity": "transaction_qty",
        "article": "product_detail",
    }
)
#add the missing columns
bakery_sales["store_id"] = None
bakery_sales["store_location"] = None
bakery_sales["product_id"] = None
bakery_sales["product_category"] = "Bakery"
bakery_sales["product_type"] = None
#reorder the columns
bakery_sales = bakery_sales[
    [
        "transaction_id",
        "transaction_date",
        "transaction_time",
        "transaction_qty",
        "store_id",
        "store_location",
        "product_id",
        "unit_price",
        "product_category",
        "product_type",
        "product_detail",
    ]
]


In [17]:
bakery_sales.to_csv("bakery_sales_new.csv", index=False)

In [18]:
unique_categories = Coffee_sales['product_category'].unique()
print(unique_categories)

['Coffee' 'Tea' 'Drinking Chocolate' 'Bakery' 'Flavours' 'Loose Tea'
 'Coffee beans' 'Packaged Chocolate' 'Branded']


In [19]:
bakery_items = Coffee_sales[Coffee_sales['product_category'] == 'Bakery'][['product_type', 'product_detail']].drop_duplicates()
print(bakery_items.nunique())


product_type       3
product_detail    11
dtype: int64


In [20]:
bakery_sales_new = pd.read_csv("bakery_sales_new.csv")
bakery_items_new = bakery_sales_new[bakery_sales_new['product_category'] == 'Bakery'][['product_detail']].drop_duplicates()
print(bakery_items_new['product_detail'].unique())


['BAGUETTE' 'PAIN AU CHOCOLAT' 'PAIN' 'TRADITIONAL BAGUETTE' 'CROISSANT'
 'BANETTE' 'BANETTINE' 'SPECIAL BREAD' 'COUPE' 'SAND JB EMMENTAL'
 'KOUIGN AMANN' 'BOULE 200G' 'BOULE 400G' 'GAL FRANGIPANE 6P' 'CAMPAGNE'
 'MOISSON' 'CAFE OU EAU' 'BRIOCHE' 'CEREAL BAGUETTE' 'SEIGLE' 'COMPLET'
 'DIVERS PATISSERIE' 'GAL FRANGIPANE 4P' 'COOKIE' 'FICELLE'
 'PAIN AUX RAISINS' 'GAL POMME 6P' 'GAL POMME 4P' 'FINANCIER X5'
 'VIK BREAD' 'DIVERS VIENNOISERIE' 'GACHE' 'SANDWICH COMPLET'
 'PAIN BANETTE' 'GRAND FAR BRETON' 'QUIM BREAD' 'SPECIAL BREAD KG'
 'GD KOUIGN AMANN' 'BOULE POLKA' 'DEMI BAGUETTE' 'CHAUSSON AUX POMMES'
 'BAGUETTE GRAINE' 'DIVERS CONFISERIE' 'SUCETTE' 'DIVERS BOULANGERIE'
 'BOISSON 33CL' 'PATES' 'FORMULE SANDWICH' 'DIVERS SANDWICHS'
 'CROISSANT AMANDES' 'PAIN CHOCO AMANDES' 'SACHET VIENNOISERIE' 'NANTAIS'
 'CHOCOLAT' 'PAIN S/SEL' 'FONDANT CHOCOLAT' 'GAL POIRE CHOCO 6P'
 'GAL POIRE CHOCO 4P' 'GALETTE 8 PERS' 'SAND JB' 'SACHET DE CROUTON'
 'GRANDE SUCETTE' 'DEMI PAIN' 'TARTELETTE' 'FLAN' '

In [21]:
def map_article_to_category(article):
    normalized = article.strip().upper()
    
    if "CAFE" in normalized:
        return "Coffee", "Coffee", article
    if normalized == "THE":
        return "Tea", "Tea", article
    
    category = "Bakery"
    
    breads = ['BAGUETTE', 'PAIN', 'BOULE', 'PAIN DE MIE', 'SEIGLE', 'VIK BREAD', 'PAIN BANETTE', 'PAIN NOIR']
    viennoiseries = ['PAIN AU CHOCOLAT', 'CROISSANT', 'KOUIGN AMANN', 'BRIOCHE', 'CHAUSSON', 'VIENNOISERIE']
    patisseries = ['TARTE', 'ECLAIR', 'FRAISIER', 'ST HONORE', 'MACARON', 'ENTREMETS', 'PARIS BREST', 'ROYAL', 'TROPEZIENNE',
                   'FINANCIER', 'CRUMBLE', 'MILLES FEUILLES', 'RELIGIEUSE', 'MERINGUE']
    sandwiches = ['SANDWICH', 'SAND JB', 'FORMULE SANDWICH']
    beverages = ['BOISSON', 'EAU']
    meals = ['PLAT', 'TRAITEUR', 'PLATEAU', 'FORMULE PLAT PREPARE', 'PLATPREPARE']
    confections = ['CONFISERIE', 'SUCETTE', 'CHOCOLAT', 'MERINGUE', 'SABLE', 'BONBON']
    
    if any(word in normalized for word in breads):
        prod_type = "Bread"
    elif any(word in normalized for word in viennoiseries):
        prod_type = "Viennoiserie"
    elif any(word in normalized for word in patisseries):
        prod_type = "Pâtisserie"
    elif any(word in normalized for word in sandwiches):
        prod_type = "Sandwich"
    elif any(word in normalized for word in meals):
        prod_type = "Meal/Traiteur"
    elif any(word in normalized for word in beverages):
        prod_type = "Beverage"
    elif any(word in normalized for word in confections):
        prod_type = "Confectionery"
    else:
        prod_type = "Other"
        
    return category, prod_type, article

bakery_sales_new[['product_category', 'product_type', 'product_detail']] = bakery_sales_new['product_detail'].apply(
    lambda x: pd.Series(map_article_to_category(x))
)

bakery_sales_new.to_csv("bakery_sales_merged.csv", index=False)


In [22]:
import pandas as pd

# Step 1: Create a mapping of product names to product IDs.
raw_product_names = [
    'BAGUETTE', 'PAIN AU CHOCOLAT', 'PAIN', 'TRADITIONAL BAGUETTE', 'CROISSANT',
    'BANETTE', 'BANETTINE', 'SPECIAL BREAD', 'COUPE', 'SAND JB EMMENTAL',
    'KOUIGN AMANN', 'BOULE 200G', 'BOULE 400G', 'GAL FRANGIPANE 6P', 'CAMPAGNE',
    'MOISSON', 'CAFE OU EAU', 'BRIOCHE', 'CEREAL BAGUETTE', 'SEIGLE', 'COMPLET',
    'DIVERS PATISSERIE', 'GAL FRANGIPANE 4P', 'COOKIE', 'FICELLE',
    'PAIN AUX RAISINS', 'GAL POMME 6P', 'GAL POMME 4P', 'FINANCIER X5',
    'VIK BREAD', 'DIVERS VIENNOISERIE', 'GACHE', 'SANDWICH COMPLET',
    'PAIN BANETTE', 'GRAND FAR BRETON', 'QUIM BREAD', 'SPECIAL BREAD KG',
    'GD KOUIGN AMANN', 'BOULE POLKA', 'DEMI BAGUETTE', 'CHAUSSON AUX POMMES',
    'BAGUETTE GRAINE', 'DIVERS CONFISERIE', 'SUCETTE', 'DIVERS BOULANGERIE',
    'BOISSON 33CL', 'PATES', 'FORMULE SANDWICH', 'DIVERS SANDWICHS',
    'CROISSANT AMANDES', 'PAIN CHOCO AMANDES', 'SACHET VIENNOISERIE', 'NANTAIS',
    'CHOCOLAT', 'PAIN S/SEL', 'FONDANT CHOCOLAT', 'GAL POIRE CHOCO 6P',
    'GAL POIRE CHOCO 4P', 'GALETTE 8 PERS', 'SAND JB', 'SACHET DE CROUTON',
    'GRANDE SUCETTE', 'DEMI PAIN', 'TARTELETTE', 'FLAN', 'PARIS BREST', 'SAVARIN',
    'FLAN ABRICOT', 'BAGUETTE APERO', 'MILLES FEUILLES', 'CHOU CHANTILLY',
    'ECLAIR', 'ROYAL 4P', 'TARTE FRUITS 6P', 'TARTE FRUITS 4P', 'NOIX JAPONAISE',
    'THE', 'BRIOCHETTE', 'ROYAL 6P', 'ECLAIR FRAISE PISTACHE', '.',
    'GD FAR BRETON', 'TRIANGLES', 'TROPEZIENNE', 'TROPEZIENNE FRAMBOISE', 'ROYAL',
    'TARTE FRAISE 6P', 'TARTELETTE FRAISE', 'TARTE FRAISE 4PER', 'FRAISIER',
    'NID DE POULE', 'TARTELETTE CHOC', 'PAIN DE MIE', 'CRUMBLE', 'FINANCIER',
    'DIVERS BOISSONS', 'CAKE', 'VIENNOISE', 'TRAITEUR', 'PAIN GRAINES',
    'PLATPREPARE6,50', 'PLATPREPARE5,50', 'PLATPREPARE7,00',
    'FORMULE PLAT PREPARE', 'ST HONORE', 'BROWNIES', 'RELIGIEUSE',
    'PLATPREPARE6,00', 'DELICETROPICAL', 'CRUMBLECARAMEL OU PISTAE',
    'PT NANTAIS', 'GD NANTAIS', 'DOUCEUR D HIVER', 'TROIS CHOCOLAT',
    'ARTICLE 295', 'TARTE FINE', 'ENTREMETS', 'BRIOCHE DE NOEL', 'FRAMBOISIER',
    'BUCHE 4PERS', 'BUCHE 6PERS', 'GD PLATEAU SALE', 'BUCHE 8PERS',
    'PT PLATEAU SALE', 'REDUCTION SUCREES 12', 'PAIN NOIR',
    'REDUCTION SUCREES 24', 'BOTTEREAU', 'MERINGUE', 'PALMIER', 'PAILLE',
    'PLAT 6.50E', 'PLAT 7.60E', 'PLAT 7.00', 'PLAT', 'PLAT 8.30E', 'FORMULE PATE',
    'GUERANDAIS', 'PALET BRETON', 'CARAMEL NOIX', 'MACARON', '12 MACARON',
    'ARMORICAIN', 'PLAQUE TARTE 25P', 'SABLE F  P', 'PAIN SUISSE PEPITO',
    'TULIPE', 'TARTELETTE COCKTAIL', 'SACHET DE VIENNOISERIE'
]

# Clean names: upper case and remove spaces.
clean_product_names = [name.strip().upper() for name in raw_product_names]

# Create a dictionary where each product name is mapped to a unique id.
product_map = {name: idx for idx, name in enumerate(clean_product_names, start=101)}

# Show the mapping information.
print("Product Mapping:")
for product, pid in product_map.items():
    print(f"{product}: {pid}")

# Read the cleaned sales file.
df = pd.read_csv('bakery_sales_merged.csv')

# Check missing values before fixing data.
print("\nMissing values before cleaning:")
print(df.isnull().sum())

# Replace missing store info with default values.
df['store_id'] = df['store_id'].fillna(1)
df['store_location'] = df['store_location'].fillna('Bakery - Default Location')

# Clean product details and format the string.
df['product_detail'] = df['product_detail'].astype(str).str.strip().str.upper()

# Ensure product_id exists and fill missing ones with 0.
if 'product_id' not in df.columns:
    df['product_id'] = 0
else:
    df['product_id'] = df['product_id'].fillna(0)

# Map the product details to product IDs using the dictionary.
df['product_id'] = df.apply(
    lambda row: product_map.get(row['product_detail'], 999) if row['product_id'] == 0 else row['product_id'],
    axis=1
)

# Convert transaction ids to integer type.
df['transaction_id'] = df['transaction_id'].astype(int)

# Show missing values after cleaning.
print("\nMissing values after cleaning:")
print(df.isnull().sum())




Product Mapping:
BAGUETTE: 101
PAIN AU CHOCOLAT: 102
PAIN: 103
TRADITIONAL BAGUETTE: 104
CROISSANT: 105
BANETTE: 106
BANETTINE: 107
SPECIAL BREAD: 108
COUPE: 109
SAND JB EMMENTAL: 110
KOUIGN AMANN: 111
BOULE 200G: 112
BOULE 400G: 113
GAL FRANGIPANE 6P: 114
CAMPAGNE: 115
MOISSON: 116
CAFE OU EAU: 117
BRIOCHE: 118
CEREAL BAGUETTE: 119
SEIGLE: 120
COMPLET: 121
DIVERS PATISSERIE: 122
GAL FRANGIPANE 4P: 123
COOKIE: 124
FICELLE: 125
PAIN AUX RAISINS: 126
GAL POMME 6P: 127
GAL POMME 4P: 128
FINANCIER X5: 129
VIK BREAD: 130
DIVERS VIENNOISERIE: 131
GACHE: 132
SANDWICH COMPLET: 133
PAIN BANETTE: 134
GRAND FAR BRETON: 135
QUIM BREAD: 136
SPECIAL BREAD KG: 137
GD KOUIGN AMANN: 138
BOULE POLKA: 139
DEMI BAGUETTE: 140
CHAUSSON AUX POMMES: 141
BAGUETTE GRAINE: 142
DIVERS CONFISERIE: 143
SUCETTE: 144
DIVERS BOULANGERIE: 145
BOISSON 33CL: 146
PATES: 147
FORMULE SANDWICH: 148
DIVERS SANDWICHS: 149
CROISSANT AMANDES: 150
PAIN CHOCO AMANDES: 151
SACHET VIENNOISERIE: 152
NANTAIS: 153
CHOCOLAT: 154
PAIN S/

In [ ]:
unique_stores = Coffee_sales[['store_id', 'store_location']].drop_duplicates()
unique_tickets = bakery_sales['transaction_id'].unique()

# Attribute a random store to each transaction
np.random.seed(42)
ticket_store_mapping = {ticket: np.random.choice(unique_stores['store_id']) for ticket in unique_tickets}

df['store_id'] = bakery_sales['transaction_id'].map(ticket_store_mapping)
df['store_location'] = df['store_id'].map(unique_stores.set_index('store_id')['store_location'])

df.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,150040,2021-01-02,08:38,1.0,3,NaN,101,0.90,Bakery,Bread,BAGUETTE
1,150040,2021-01-02,08:38,3.0,3,NaN,102,1.20,Bakery,Bread,PAIN AU CHOCOLAT
2,150041,2021-01-02,09:14,2.0,5,NaN,102,1.20,Bakery,Bread,PAIN AU CHOCOLAT
3,150041,2021-01-02,09:14,1.0,5,NaN,103,1.15,Bakery,Bread,PAIN
4,150042,2021-01-02,09:25,5.0,3,NaN,104,1.20,Bakery,Bread,TRADITIONAL BAGUETTE


In [24]:
# Save the cleaned data to a csv file.
output_filename = 'Bakery_sales_cleaned.csv'
df.to_csv(output_filename, index=False)
print(f"\nCleaning complete. Cleaned data saved as '{output_filename}'.")


Cleaning complete. Cleaned data saved as 'Bakery_sales_cleaned.csv'.


In [25]:
df = pd.read_csv("Bakery_sales_cleaned.csv")
df['sales_id'] = range(1, len(df) + 1)
df.to_csv("Bakery_sales_cleaned.csv", index=False)

df2 = pd.read_csv("Coffee_Shop_Sales.csv") 
df2['sales_id'] = range(1, len(df2) + 1)
df2.to_csv("Coffee_Shop_Sales.csv", index=False)

print(df['sales_id'].head())
print(df2['sales_id'].head())

0    1
1    2
2    3
3    4
4    5
Name: sales_id, dtype: int64
0    1
1    2
2    3
3    4
4    5
Name: sales_id, dtype: int64


![Local Image](erd.png)


In [26]:
CREATE TABLE Store (
    store_id INT PRIMARY KEY,
    store_location VARCHAR(255)
);

CREATE TABLE Transactions (
    sale_id INT PRIMARY KEY,
    transaction_id INT,
    transaction_date DATE,
    transaction_time TIME,
    transaction_qty INT,
    store_id INT,
    product_id INT,
    FOREIGN KEY (store_id) REFERENCES Store(store_id),
    FOREIGN KEY (product_id) REFERENCES Product(product_id)
);

CREATE TABLE Product (
    product_id INT PRIMARY KEY,
    product_category VARCHAR(255) NOT NULL,  
    product_type VARCHAR(255),              
    product_detail VARCHAR(255),             
    unit_price DECIMAL(10,2) NOT NULL
);


SyntaxError: invalid syntax (2191229633.py, line 1)